## To do:

Handle games where players transfer servers

# Notebook for scraping Korea

I created a separate notebook to scrape different regions. This allows me to collect games faster.

Load libraries

In [1]:
import requests, json
import numpy as np
from lolML.src import API_io
import importlib
import pandas as pd
import time
from lolML.src import feature_calc

In [2]:
working_dir = 'C:\\Users\\palmiteradmin\\Documents\\GitHub\\lolML'
with open(working_dir+ '\\api_key.txt', 'r') as api_file:
    api_key =  api_file.read()

## Get a list of high level matches

Get summoners in featured games

In [3]:
region_key = 'euw'
featured_json = API_io.load_featured_games(api_key, region_key) # load json of featured games
featured_game_ids = [x['gameId'] for x in featured_json ] #  use list comprehension to get featured games; don't use this

Make a list of summoner names and summoner IDs from the featured JSON

In [4]:
summoner_names, summoner_IDs = API_io.get_summoners_IDs_from_featured_games(featured_json, api_key, region =region_key)
summoner_names[:5]

['Sunnyyyyyyyyyyy', 'Westboro', 'Tito Werlyb', 'Traxx', 'ArKaDaTa']

In [9]:
summoner_urls = [API_io.make_matchlist_url_summoner_ID(x, True, True, api_key, region =region_key) for x in summoner_IDs]
print(summoner_urls[:2])
match_histories = [API_io.get_limited_request(x) for x in summoner_urls ]

['https://euw.api.pvp.net/api/lol/euw/v2.2/matchlist/by-summoner/63913386?rankedQueues=RANKED_SOLO_5x5&seasons=SEASON2015&api_key=0da3703d-7bf5-4e72-96cd-5062b28720d7', 'https://euw.api.pvp.net/api/lol/euw/v2.2/matchlist/by-summoner/38513158?rankedQueues=RANKED_SOLO_5x5&seasons=SEASON2015&api_key=0da3703d-7bf5-4e72-96cd-5062b28720d7']


Extract the match ID from the match history JSON

In [20]:
match_IDs = np.empty(0, dtype=int)
for cur_matches in match_histories:
    match_IDs = np.append( match_IDs, [x['matchId'] for x in cur_matches['matches'] if x['region'] == region_key.upper()] )
match_IDs = np.unique(match_IDs)
pd.Series(match_IDs).to_csv('Korean Match IDs.csv')
match_IDs.shape

(28056,)

In [21]:
# make urls for loading
match_urls = [API_io.make_match_info_url(x, True, api_key, region = region_key) for x in match_IDs] # True flag means we get the timeline
match_urls[:2]

['https://euw.api.pvp.net/api/lol/euw/v2.2/match/1930396612?includeTimeline=true&api_key=0da3703d-7bf5-4e72-96cd-5062b28720d7',
 'https://euw.api.pvp.net/api/lol/euw/v2.2/match/1930399900?includeTimeline=true&api_key=0da3703d-7bf5-4e72-96cd-5062b28720d7']

In [94]:
match_range = np.arange(5000, 10000)
col_names = feature_calc.col_names
timeline_end = 55
time_indices = np.arange(5, timeline_end, 5)
games_df = pd.DataFrame(index = match_range, columns= col_names)
timelines_df = [ pd.DataFrame(index = match_range, columns= col_names) for x in time_indices]

In [ ]:
# this is the worst code EVER
for match_index, cur_match in enumerate(match_range):
    try:
        match_info = requests.get(match_urls[cur_match]).json()
        for time_index, last_min in enumerate(time_indices):
            try:
                timelines_df[time_index].loc[match_index] = feature_calc.calc_features_single_match(match_info, last_min)
            except:
                continue
    except requests.exceptions.HTTPError as err:
        print('HHTPError in game ' + str(match_IDs[cur_match]))
    except:
        import sys
        err = sys.exc_info()[0]
        print('Error: ' + str(err) + ' in game ' + str(match_IDs[cur_match]))
    time.sleep(0.3) # RIOT API is throttled to ~0.83 requests / second

In [118]:
timelines_df = [x.dropna() for x in timelines_df]
timelines_df = [feature_calc.retype_columns(x) for x in timelines_df ]
#timelines_df = [x.set_index('matchId') for x in timelines_df ]

In [119]:
[x.shape for x in timelines_df]

[(4914, 31),
 (4908, 31),
 (4867, 31),
 (4761, 31),
 (4028, 31),
 (3139, 31),
 (1960, 31),
 (940, 31),
 (345, 31),
 (87, 31)]

Now save the information to disk.

In [59]:
games_df.to_csv('Korean games 1-1000.csv')

In [98]:
combined_df = [timelines_df[x].append(combined_df[x] ) for x in np.arange(len(timelines_df)) ]

ValueError: incompatible categories in categorical concat

In [125]:
importlib.reload(feature_calc)
timelines_df = [feature_calc.retype_columns(x) for x in timelines_df ]
timelines_df[0].dtypes

first_dragon    category
drag_diff          int64
total_drag         int64
first_baron     category
blue_barons        int64
red_barons         int64
first_tower     category
tower_diff         int64
total_tower        int64
first_inhib     category
blue_inhibs        int64
red_inhibs         int64
first_blood     category
gold_diff          int64
kill_diff          int64
total_kill         int64
blue_share         int64
red_share        float64
blue_0             int64
blue_1             int64
blue_2             int64
blue_3             int64
blue_4             int64
red_0              int64
red_1              int64
red_2              int64
red_3              int64
red_4              int64
surrender       category
game_length        int64
winner          category
dtype: object